#Fake News Detection with BERT using News Content and User Engagement features

Here, we'll train a model to predict whether a news article is fake or not using features from the news articles as well as user engagement data from Twitter. Code was adapted from [this colab notebook](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb).

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 5.7MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'model_out3'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: model_out3 *****


#Data Loading

In [0]:
def strconcat(series):
  return series.str.cat(sep = '. ')

In [0]:
def retfirst(series):
  return series.iloc[0]

In [0]:
def categorize_label(series):
  m = series.mean()
  return 1 if m > 0.5 else 0 if m < 0.5 else np.nan

In [0]:
from tensorflow import keras
import os
import re
from google.colab import drive

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  drive.mount('/content/drive')
  unpickled_df = pd.read_pickle('/content/drive/My Drive/FakeNewsNet-master/Downloaded Datasets/fakenewsnet_dataset_tweets.pkl')
  # Drop rows with missing news info.
  unpickled_df['news_source'].replace('', np.nan, inplace=True)
  unpickled_df['news_text'].replace('', np.nan, inplace=True)
  unpickled_df = unpickled_df.dropna(subset=['news_source', 'news_text'])
  # Drop user PII columns.
  unpickled_df.drop(columns=['user_screen_name', 'user_id', 'user_location'], inplace=True)
  # Group by news id and aggregate the remaining columns appropriately.
  grouped_df = unpickled_df.groupby(['news_id']).agg({
      'tweet' : [strconcat], 
      'label' : [categorize_label],
      'user_verified' : [categorize_label],
      'user_friends_count' : ['mean'],
      'user_followers_count' : ['mean'],
      'user_listed_count': ['mean'],
      'user_statuses_count': ['mean'],
      'user_favourites_count': ['mean'],
      'title': [retfirst],
      'news_source': [retfirst],
      'news_text': [retfirst],
    })
  # Rename columns since the grouping and aggregation results in tuples as
  # column names..
  grouped_df.columns = ['tweet', 'label', 'user_verified', 'user_friends_count',
                      'user_followers_count', 'user_listed_count', 'user_statuses_count',
                      'user_favourites_count', 'title', 'news_source', 'news_text']
  # Recast label as bool.
  grouped_df['label'] = grouped_df['label'].astype(bool)
  # Recast user_verified as int.
  grouped_df = grouped_df.dropna(subset=['user_verified'])
  grouped_df['user_verified'] = grouped_df['user_verified'].astype(int)

  # Split into train/dev/test sets.
  grouped_df = grouped_df.sample(frac=1).reset_index(drop=True)
  train_df = grouped_df[0:18000]
  dev_df = grouped_df[18000:19000]
  test_df = grouped_df[19000:]

  return train_df, dev_df, test_df


In [0]:
train, dev, test = download_and_load_datasets()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


For us, our input data is the 'text' and 'titile' columns and our label is the 'label' column.

In [0]:
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [True, False]

#Data Preprocessing

##Data Preprocessing for BERT
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['title'] + x['news_text'] + x['tweet'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

dev_InputExamples = dev.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['title'] + x['news_text'] + x['tweet'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['title'] + x['news_text'] + x['tweet'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [0]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer!! Here is the text.")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer',
 '!',
 '!',
 'here',
 'is',
 'the',
 'text',
 '.']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
dev_features = bert.run_classifier.convert_examples_to_features(dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 18000


INFO:tensorflow:Writing example 0 of 18000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: False (id = 1)


INFO:tensorflow:label: False (id = 1)


##Creating an Input Generator

Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
def input_fn_builder(dataframe, features, seq_length, is_training, drop_remainder):

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  news_source = []
  user_verified = []
  user_friends_count = []
  user_followers_count = []
  user_listed_count = []
  user_statuses_count = []
  user_favourites_count = []

  # Feature gen preserves the order so we can iterate over dataframe and
  # feature together.
  for i, feature in enumerate(features):
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

    row = dataframe.iloc[i]
    news_source.append(row.news_source)
    user_verified.append(row.user_verified)
    user_friends_count.append(row.user_friends_count)
    user_followers_count.append(row.user_followers_count)
    user_listed_count.append(row.user_listed_count)
    user_statuses_count.append(row.user_statuses_count)
    user_favourites_count.append(row.user_favourites_count)

  news_source_categorized, _ = pd.factorize(news_source)
  news_source_categorized = news_source_categorized.tolist()

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),

        "news_source":
            tf.constant(news_source_categorized, shape=[num_examples], dtype=tf.float32),
        "user_verified":
            tf.constant(user_verified, shape=[num_examples], dtype=tf.float32),
        "user_friends_count":
            tf.constant(user_friends_count, shape=[num_examples], dtype=tf.float32),
        "user_followers_count":
            tf.constant(user_followers_count, shape=[num_examples], dtype=tf.float32),
        "user_listed_count":
            tf.constant(user_listed_count, shape=[num_examples], dtype=tf.float32),
        "user_statuses_count":
            tf.constant(user_statuses_count, shape=[num_examples], dtype=tf.float32),
        "user_favourites_count":
            tf.constant(user_favourites_count, shape=[num_examples], dtype=tf.float32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn    

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = input_fn_builder(
    dataframe=train,
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
# Compute train and warmup steps from batch size
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_TRAIN_EPOCHS = 10.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100
DROPOUT_PROB = 0.8

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids,
                 news_source, user_verified, user_followers_count,
                 user_listed_count, user_statuses_count, user_favourites_count,
                 labels, num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)


  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  
  hidden_size = output_layer.shape[-1].value  

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())
  
  # Create 7 extra layers for the output from BERT and the other non-BERT
  # inputs. We use the same number of hidden units as in the last BERT layer
  # for now. But this architecture hyperparameter can be tuned.
  layer1_weights = tf.get_variable(
      "layer1_weights", [hidden_size, 7], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  layer1_bias = tf.get_variable(
      "layer1_bias", [hidden_size], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  layer2_weights = tf.get_variable(
      "layer2_weights", [hidden_size, hidden_size], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  layer2_bias = tf.get_variable(
      "layer2_bias", [hidden_size], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  layer3_weights = tf.get_variable(
      "layer3_weights", [hidden_size, hidden_size], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  layer3_bias = tf.get_variable(
      "layer3_bias", [hidden_size], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  layer4_weights = tf.get_variable(
      "layer4_weights", [hidden_size, hidden_size], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  layer4_bias = tf.get_variable(
      "layer4_bias", [hidden_size], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  layer5_weights = tf.get_variable(
      "layer5_weights", [hidden_size, hidden_size], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  layer5_bias = tf.get_variable(
      "layer5_bias", [hidden_size], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  layer6_weights = tf.get_variable(
      "layer6_weights", [hidden_size, hidden_size], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  layer6_bias = tf.get_variable(
      "layer6_bias", [hidden_size], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  final_output_weights = tf.get_variable(
      "final_output_weights", [num_labels, hidden_size], dtype=tf.float32,
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  final_output_bias = tf.get_variable(
      "final_output_bias", [num_labels], dtype=tf.float32, initializer=tf.zeros_initializer())
  
  with tf.variable_scope("loss"):
    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    bert_logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    bert_logits = tf.nn.bias_add(bert_logits, output_bias)
    bert_log_probs = tf.nn.log_softmax(bert_logits, axis=-1)

    # Use low level api.
    bert_prediction = tf.squeeze(tf.argmax(bert_log_probs, axis=-1, output_type=tf.int32))
    bert_prediction = tf.cast(bert_prediction, tf.float32)
    dnn_inputs = tf.stack(
      [news_source, user_verified, user_followers_count, user_listed_count,
       user_statuses_count, user_favourites_count, bert_prediction], 0)
    
    dnn_layer1 = tf.matmul(dnn_inputs, layer1_weights, transpose_a=True, transpose_b=True)
    dnn_layer1 = tf.nn.bias_add(dnn_layer1, layer1_bias)
    dnn_layer1 = tf.nn.leaky_relu(dnn_layer1, alpha=0.2, name="dnn_layer_1")
    dnn_layer1 = tf.nn.dropout(dnn_layer1, keep_prob=DROPOUT_PROB)
  
    dnn_layer2 = tf.matmul(dnn_layer1, layer2_weights, transpose_b=True)
    dnn_layer2 = tf.nn.bias_add(dnn_layer2, layer2_bias)
    dnn_layer2 = tf.nn.leaky_relu(dnn_layer2, alpha=0.2, name="dnn_layer_2")
    dnn_layer2 = tf.nn.dropout(dnn_layer2, keep_prob=DROPOUT_PROB)

    dnn_layer3 = tf.matmul(dnn_layer2, layer3_weights, transpose_b=True)
    dnn_layer3 = tf.nn.bias_add(dnn_layer3, layer3_bias)
    dnn_layer3 = tf.nn.leaky_relu(dnn_layer3, alpha=0.2, name="dnn_layer_3")
    dnn_layer3 = tf.nn.dropout(dnn_layer3, keep_prob=DROPOUT_PROB)

    dnn_layer4 = tf.matmul(dnn_layer3, layer4_weights, transpose_b=True)
    dnn_layer4 = tf.nn.bias_add(dnn_layer4, layer4_bias)
    dnn_layer4 = tf.nn.leaky_relu(dnn_layer4, alpha=0.2, name="dnn_layer_4")
    dnn_layer4 = tf.nn.dropout(dnn_layer4, keep_prob=DROPOUT_PROB)

    dnn_layer5 = tf.matmul(dnn_layer4, layer5_weights, transpose_b=True)
    dnn_layer5 = tf.nn.bias_add(dnn_layer5, layer5_bias)
    dnn_layer5 = tf.nn.leaky_relu(dnn_layer5, alpha=0.2, name="dnn_layer_5")
    dnn_layer5 = tf.nn.dropout(dnn_layer5, keep_prob=DROPOUT_PROB)

    dnn_layer6 = tf.matmul(dnn_layer5, layer6_weights, transpose_b=True)
    dnn_layer6 = tf.nn.bias_add(dnn_layer6, layer6_bias)
    dnn_layer6 = tf.nn.leaky_relu(dnn_layer6, alpha=0.2, name="dnn_layer_6")
    dnn_layer6 = tf.nn.dropout(dnn_layer6, keep_prob=DROPOUT_PROB)

    # Final softmax layer
    logits = tf.matmul(dnn_layer6, final_output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, final_output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, etc.
def model_fn_builder(learning_rate, num_labels, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    # Features that don't go into BERT but go into the DNN layers.
    news_source = features["news_source"]
    user_verified = features["user_verified"]
    user_followers_count = features["user_followers_count"]
    user_listed_count = features["user_listed_count"]
    user_statuses_count = features["user_statuses_count"]
    user_favourites_count = features["user_favourites_count"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, 
        news_source, user_verified, user_followers_count, user_listed_count,
        user_statuses_count, user_favourites_count, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg,
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  learning_rate=LEARNING_RATE,
  num_labels=len(label_list),
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'model_out3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7909836b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'model_out3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7909836b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Now we train our model! 

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 5625 into model_out3/model.ckpt.


INFO:tensorflow:Saving checkpoints for 5625 into model_out3/model.ckpt.


INFO:tensorflow:Loss for final step: 0.22800115.


INFO:tensorflow:Loss for final step: 0.22800115.


Training took time  0:15:32.887869


#Evaluate on the Dev set

Now let's use our dev data to see how well our model did:

In [0]:
dev_input_fn = input_fn_builder(
    dataframe=dev,
    features=dev_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=dev_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5625: auc = 0.7589625, eval_accuracy = 0.859, f1_score = 0.9113765, false_negatives = 40.0, false_positives = 101.0, global_step = 5625, loss = 0.37621534, precision = 0.877724, recall = 0.9477124, true_negatives = 134.0, true_positives = 725.0


INFO:tensorflow:Saving dict for global step 5625: auc = 0.7589625, eval_accuracy = 0.859, f1_score = 0.9113765, false_negatives = 40.0, false_positives = 101.0, global_step = 5625, loss = 0.37621534, precision = 0.877724, recall = 0.9477124, true_negatives = 134.0, true_positives = 725.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5625: model_out3/model.ckpt-5625


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5625: model_out3/model.ckpt-5625


{'auc': 0.7589625,
 'eval_accuracy': 0.859,
 'f1_score': 0.9113765,
 'false_negatives': 40.0,
 'false_positives': 101.0,
 'global_step': 5625,
 'loss': 0.37621534,
 'precision': 0.877724,
 'recall': 0.9477124,
 'true_negatives': 134.0,
 'true_positives': 725.0}

#Evaluate on the test Set

In [0]:
test_input_fn = input_fn_builder(
    dataframe=test,
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5625: auc = 0.7443611, eval_accuracy = 0.8416816, f1_score = 0.89948887, false_negatives = 59.0, false_positives = 118.0, global_step = 5625, loss = 0.42407137, precision = 0.8703297, recall = 0.9306698, true_negatives = 149.0, true_positives = 792.0


INFO:tensorflow:Saving dict for global step 5625: auc = 0.7443611, eval_accuracy = 0.8416816, f1_score = 0.89948887, false_negatives = 59.0, false_positives = 118.0, global_step = 5625, loss = 0.42407137, precision = 0.8703297, recall = 0.9306698, true_negatives = 149.0, true_positives = 792.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5625: model_out2/model.ckpt-5625


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5625: model_out2/model.ckpt-5625


{'auc': 0.7443611,
 'eval_accuracy': 0.8416816,
 'f1_score': 0.89948887,
 'false_negatives': 59.0,
 'false_positives': 118.0,
 'global_step': 5625,
 'loss': 0.42407137,
 'precision': 0.8703297,
 'recall': 0.9306698,
 'true_negatives': 149.0,
 'true_positives': 792.0}